In [1]:
from glob import glob
import os.path as osp
import os
import pickle
import json
import numpy as np

In [2]:
# load GT
with open('../data/export/tapos_annotation_timestamps_myfps.json', 'r') as f:
    tapos_dict = json.load(f)
    
#only need go over action instances that have at least 1 internal boundary    
num_bdy_per_action = []
num_frames_per_action = []
time_duration_per_action = []
tapos_dict_val_instance_level_at_least_1_internal_bdy = {}
for k in tapos_dict.keys():
    for s in tapos_dict[k].keys():
        if tapos_dict[k][s]['subset'] == 'val':
            if len(tapos_dict[k][s]['substages']) > 2:
                num_bdy_per_action.append(len(tapos_dict[k][s]['substages'])-2)
                num_frames_per_action.append(tapos_dict[k][s]['substages'][-1]+1-tapos_dict[k][s]['substages'][0])
                time_duration_per_action.append(tapos_dict[k][s]['substages_timestamps'][-1]-tapos_dict[k][s]['substages_timestamps'][0])
                tapos_dict_val_instance_level_at_least_1_internal_bdy[k+'_'+s] = tapos_dict[k][s]
                tapos_dict_val_instance_level_at_least_1_internal_bdy[k+'_'+s]['video_name'] = k
                tapos_dict_val_instance_level_at_least_1_internal_bdy[k+'_'+s]['shot_name'] = s

print("total number of boundaries: " + str(sum(num_bdy_per_action)))
print("total number of frames: " + str(sum(num_frames_per_action)))

total number of boundaries: 3312
total number of frames: 436888


In [3]:
downsample = 3

exp_path = '../data/exp_TAPOS/'
output_seg_dir = 'detect_seg'
OUTPUT_BDY_PATH = exp_path + output_seg_dir + '/{}.pkl'

In [4]:
list_rec = []
list_prec = []
list_f1 = []

for d in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:

    tp_all = 0
    num_pos_all = 0
    num_det_all = 0
    
    for vid_id in list(tapos_dict_val_instance_level_at_least_1_internal_bdy.keys()):

        output_bdy_path = OUTPUT_BDY_PATH.format(vid_id)

        if not os.path.exists(output_bdy_path):
            num_pos_all += len(tapos_dict_val_instance_level_at_least_1_internal_bdy[vid_id]['substages_myframeidx'])-2
            continue
        
        with open(output_bdy_path, 'rb') as f:
            bdy_idx_save = pickle.load(f, encoding='latin1') 

        bdy_idx_list_smt = np.array(bdy_idx_save['bdy_idx_list_smt'])*downsample # already offset, index starts from 0

        v = vid_id[:11]
        s = vid_id[12:]
        myfps = tapos_dict[v][s]['myfps']
        bdy_idx_list_gt = tapos_dict[v][s]['substages_myframeidx'][1:-1]
        
        ins_start = tapos_dict[v][s]['substages_myframeidx'][0]
        ins_end = tapos_dict[v][s]['substages_myframeidx'][-1]
        
        if bdy_idx_list_gt == []:
            continue
            
        num_pos = len(bdy_idx_list_gt)
        num_pos_all += num_pos
        
        # remove detected boundary outside the action instance
        tmp = []
        for det in bdy_idx_list_smt:
            tmpdet = det + tapos_dict[v][s]['substages_myframeidx'][0]
            if tmpdet >= (tapos_dict[v][s]['substages_myframeidx'][0]) and tmpdet <= (tapos_dict[v][s]['substages_myframeidx'][-1]):
                tmp.append(tmpdet)
        bdy_idx_list_smt = tmp
        
        if bdy_idx_list_smt == []:
            continue
            
        # compare bdy_idx_list_smt vs. bdy_idx_list_gt
        
        num_det = len(bdy_idx_list_smt)
        num_det_all += num_det
        tp = 0
        offset_arr = np.zeros((len(bdy_idx_list_gt), len(bdy_idx_list_smt))) 
        
        for ann1_idx in range(len(bdy_idx_list_gt)):
            for ann2_idx in range(len(bdy_idx_list_smt)):
                offset_arr[ann1_idx, ann2_idx] = abs(bdy_idx_list_gt[ann1_idx]-bdy_idx_list_smt[ann2_idx])
        
        for ann1_idx in range(len(bdy_idx_list_gt)):
            if offset_arr.shape[1] == 0:
                break
            min_idx = np.argmin(offset_arr[ann1_idx, :])
            if offset_arr[ann1_idx, min_idx] <= d*(ins_end-ins_start+1):
                tp += 1
                offset_arr = np.delete(offset_arr, min_idx, 1)   
                
        tp_all += tp
        
    fn_all = num_pos_all - tp_all
    fp_all = num_det_all - tp_all
    if num_pos_all == 0:
        rec = 1
    else:
        rec = tp_all/(tp_all+fn_all)
    if (tp_all+fp_all) == 0:
        prec = 0
    else:
        prec = tp_all/(tp_all+fp_all)
    if (rec+prec) == 0:
        f1 = 0
    else:
        f1 = 2*rec*prec/(rec+prec)
    list_rec.append(rec); list_prec.append(prec); list_f1.append(f1)

print("rec: " + str(np.mean(list_rec))) 
print("prec: " + str(np.mean(list_prec))) 
print("F1: " + str(np.mean(list_f1))) 

print("rec: " + str(list_rec))
print("prec: " + str(list_prec))
print("F1: " + str(list_f1)) 
    
np.save(exp_path + output_seg_dir + '.eval.rgb.npy', [list_rec, list_prec, list_f1]) 


rec: 0.4321256038647343
prec: 0.7032923832923832
F1: 0.535328221432579
rec: [0.27747584541062803, 0.3565821256038647, 0.39945652173913043, 0.427536231884058, 0.45018115942028986, 0.4643719806763285, 0.47493961352657005, 0.4830917874396135, 0.4906400966183575, 0.4969806763285024]
prec: [0.4515970515970516, 0.5803439803439804, 0.6501228501228501, 0.6958230958230959, 0.7326781326781326, 0.7557739557739558, 0.772972972972973, 0.7862407862407862, 0.7985257985257985, 0.8088452088452088]
F1: [0.3437441556012717, 0.4417430334767159, 0.4948569291191322, 0.5296427903497288, 0.5576959042453713, 0.5752758556199739, 0.5883673087712735, 0.5984664297737048, 0.6078174677389191, 0.6156723396296989]


#### previous code on evaluating PA mistakenly used `bdy_idx_list_smt = bdy_idx_save['bdy_idx_list_smt']*downsample`, where `bdy_idx_save['bdy_idx_list_smt']` is a list in python. This should be corrected to `np.array(bdy_idx_save['bdy_idx_list_smt'])*downsample`.

Previous results (note that this is wrong):
```python
rec: 0.3724033816425121

prec: 0.19957928802588995

F1: 0.259882005899705

rec: [0.1766304347826087, 0.22433574879227053, 0.2660024154589372, 0.3052536231884058, 0.34541062801932365, 0.3888888888888889, 0.43719806763285024, 0.4815821256038647, 0.5256642512077294, 0.5730676328502415]

prec: [0.09466019417475728, 0.12022653721682848, 0.14255663430420712, 0.16359223300970874, 0.18511326860841423, 0.20841423948220064, 0.2343042071197411, 0.2580906148867314, 0.28171521035598707, 0.30711974110032364]

F1: [0.12326169405815424, 0.15655288664138223, 0.185630004214075, 0.21302149178255372, 0.24104509060261273, 0.2713864306784661, 0.3050990307627475, 0.3360724820901812, 0.3668352296670881, 0.3999157184997893]
```